In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# MY WORK
# AGE and CHILDREN convert from int64 to float64 bcs NN better works with floating-point numbers
dataset['age']=dataset['age'].astype('float64')
dataset['children']=dataset['children'].astype('float64')
# SEX and SMOKER map from categorical to numerical columns as float64, they are already normalized just 0 and 1 values
dataset['sex'] = dataset['sex'].map({'female':0,'male':1}).astype('float64')
dataset['smoker'] = dataset['smoker'].map({'yes':1,'no':0}).astype('float64')
# one hot encoding on region column to convert categorical into numerical, normalized 0&1
# region1 = 0 at all other regions
dataset = pd.get_dummies(dataset,columns=['region'],drop_first = True)
TO_NORMALIZE = ['age','bmi','children','region']
# spliting dataset to train and test sets, 80:20 and also seed=0
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
# taking labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')
# making normalization layer for dataset
norm = tf.keras.layers.Normalization(axis=-1)
norm.adapt(train_dataset)
# making architecture
model = keras.Sequential([
      norm,
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(1)
  ])
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'mean_absolute_error',
    metrics=['mae','mse']
)
history = model.fit(
    train_dataset,
    train_labels,
    epochs = 100,
    verbose=0,
    validation_split=0.2
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
